<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/04_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# colab session setup
%cd /content
!rm -rf mimic-readmission-analysis
!git clone https://github.com/seoyeon0905/mimic-readmission-analysis.git
%cd mimic-readmission-analysis

/content
Cloning into 'mimic-readmission-analysis'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 188 (delta 20), reused 5 (delta 5), pack-reused 155 (from 1)
Receiving objects: 100% (188/188), 493.86 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/mimic-readmission-analysis


In [3]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# 04_baseline_model.ipynb의 목표
- EDA에서 확인한 사실을 바탕으로 단순하지만 정직한 예측 모델을 만들고 올바른 평가 지표로 성능을 해석
- EDA에서 본 사실을 바탕으로 단순한 모델이 어디까지 가능한지를 수치로 보여주는 기준점(baseline) 만들기

# 04. Baseline 예측 모델링

이 노트북에서는 기본적인 인구학적 및 임상 변수만을 사용하여
30일 이내 ICU 재입원을 얼마나 잘 예측할 수 있는지 평가하기 위한
간단한 기준선(baseline) 예측 모델을 구축한다.

탐색적 데이터 분석(EDA) 결과를 바탕으로,
단일 변수만으로는 예측력이 제한적일 것으로 예상하며,
이후 단계의 모델링 성능을 비교하기 위한
참조 성능(reference performance)을 설정하는 데 목적이 있다.

# 데이터 로드(EDA와 동일한 csv)

In [4]:
# 작업위치(폴더)를 notebooks로 변경하기
# 그 작업위치(폴더)에 어떤 파일이 있는지 확인하기
%cd notebooks
!ls

/content/mimic-readmission-analysis/notebooks
01_data_loading.ipynb	    03_readmission_eda.ipynb
02_cohort_definition.ipynb  04_baseline_model.ipynb


In [5]:
# 데이터 로드(EDA와 동일한 csv)
df = pd.read_csv("../data/processed/first_icu_admissions.csv")

# Baseline feature 선택(중요한 판단 포인트)
## EDA 결과를 그대로 반영하면:
- age -> 약한 정보
- icu_los -> 약한 positive signal
- gender -> 거의 없음(baseline에서는 제외해도 됨)

In [6]:
# X, y만들기(모델 입력 / 정답 만들기)
# 모델에게 줄 "힌트(X)"와 "정답(y)"를 분리해야 학습/평가가 가능
# Feature / Target 설정

features = ['age', 'icu_los']
target = 'readmitted_30d'

x = df[features]
y = df[target]

In [7]:
print("x shape:", x.shape)
print("y shape:", y.shape)
print("Target distribution:\n", y.value_counts())
# x shape: (100, 2)
# y shape: (100,)
# y 분포가 0:92, 1:8 정도 나오면 정상

x shape: (100, 2)
y shape: (100,)
Target distribution:
 readmitted_30d
0    92
1     8
Name: count, dtype: int64


# Train/Test split(공부 / 시험 데이터 나누기)
모델이 외운 실력이 아니라 처음 보는 데이터에 대한 실력을 평가하려고

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size = 0.3,  # 30%는 시험용
    random_state = 42,  # 결과 고정(재현성)
    stratify = y  # 불균형 유지
)

In [9]:
print("Train size:", x_train.shape[0])
print("Test size:", x_test.shape[0])

print("\nTrain label distribution:\n", y_train.value_counts())
print("\nTest label distribution:\n", y_test.value_counts())
# Train size: 70, Test size: 30
# Test에도 1이 최소 1개 이상 있어야 정상
# 없으면 stratify가 제대로 안 된 거라 다시 봐야 함

Train size: 70
Test size: 30

Train label distribution:
 readmitted_30d
0    64
1     6
Name: count, dtype: int64

Test label distribution:
 readmitted_30d
0    28
1     2
Name: count, dtype: int64


# Navie baseline(전부 0으로 찍어보기)
아무 모델도 안 써도 나올 수 있는 성능이 어느 정도인지 기준점을 만들기 위해서

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# 전부 0으로 예측
y_pred_naive = np.zeros_like(y_test)

In [11]:
print("=== Naive Baseline (all 0) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_naive))
print("Recall:", recall_score(y_test, y_pred_naive, zero_division=0))
print("Precision:", precision_score(y_test, y_pred_naive, zero_division=0))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_naive))
# Accuracy는 높게 나올 수 있음(0이 많으니까)
# Recall은 0일 가능성이 큼(1을 하나도 못 맞춤)
# Confusion Matrix에서 TP=0, FN>0이면 "재입원 환자를 못 잡음"의미

=== Naive Baseline (all 0) ===
Accuracy: 0.9333333333333333
Recall: 0.0
Precision: 0.0

Confusion Matrix:
 [[28  0]
 [ 2  0]]


# Logistic Regression 학습하기(진짜 baseline 모델)
- 모델링 처음 단계에서 가장 "기본 + 해석 가능"
- 불균형도 class_weight = 'balanced'로 어느 정도 대응 가능

In [12]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    class_weight = 'balanced',
    max_iter = 1000
)

log_reg.fit(x_train, y_train)

# 예측(0/1)
y_pred = log_reg.predict(x_test)

# 확률점수(재입원=1일 확률)
y_proba = log_reg.predict_proba(x_test)[:, 1]

In [13]:
print("Logistic Regression fitted.")
print("Example probabilities:", y_proba[:10])
# Example probabilities에서 0~1 사이 값들이 나오면 정상

Logistic Regression fitted.
Example probabilities: [0.48118874 0.36397206 0.49600107 0.36257137 0.48978579 0.48721873
 0.46876771 0.44836183 0.49244947 0.28909225]


# 평가 지표 출력(여기가 핵심)
불균형 문제에서 accuracy만 보면 착시가 생기니까 Recall / AUROC / PR-AUC를 함께 봐야함

In [14]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [16]:
print("=== Logistic Regression Performance ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, zero_division=0))
print("Recall:", recall_score(y_test, y_pred, zero_division=0))

print("AUROC:", roc_auc_score(y_test, y_proba))
print("PR-AUC:", average_precision_score(y_test, y_proba))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
# 데이터가 1이 적기때문에
# AUROC가 0.5보다 조금만 높아도 "약한 신호"
# PR-AUC는 낮게 나오는 게 정상(rare event니까)

=== Logistic Regression Performance ===
Accuracy: 0.8666666666666667
Precision: 0.0
Recall: 0.0
AUROC: 0.4821428571428572
PR-AUC: 0.09109311740890688

Confusion Matrix:
 [[26  2]
 [ 2  0]]


# ROC / PR Curve 그리기(시각화)